# Intro to plotly

Probably for a separate tutorial session...

In [1]:
import numpy as np
import pandas as pd

In [2]:
from plotly import __version__
# from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.offline as py
import plotly.graph_objs as go

In [3]:
print(__version__)

3.5.0


In [4]:
py.init_notebook_mode(connected=True)

There are two basic plotting methods:

* `py.plot` generates HTML that is saved locally and viewed in your browser
* `py.iplot` generates HTML that is displayed in a notebook

In [5]:
# When working in a notebook, use iplot rather than plot
py.iplot([go.Scatter(x=[1, 2, 3], y=[3, 1, 6])])

In [6]:
trace0 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[10, 15, 13, 17]
)
trace1 = go.Scatter(
    x=[1, 2, 3, 4],
    y=[16, 5, 11, 9]
)
data = [trace0, trace1]

In [7]:
py.iplot(data)

### Cufflinks allows you to use pandas DataFrames directly with plotly

In [8]:
import cufflinks as cf

In [9]:
df = cf.datagen.lines()
df.head()

,ZBV.JY,ETH.JG,ZXX.GW,AZK.KB,CVQ.RQ
2015-01-01,0.611363,-1.096056,-0.629645,1.143079,-0.170742
2015-01-02,1.551859,-1.055268,-2.351348,1.567042,1.428842
2015-01-03,2.674079,-0.182337,-2.104389,0.668312,0.415188
2015-01-04,3.802940,-1.505826,-1.976627,2.437671,0.215318
2015-01-05,4.957813,-2.515072,-0.233860,3.733720,-0.567662


In [10]:
py.iplot(df.iplot(asFigure=True, kind='scatter', xTitle='Dates', yTitle='Returns', title='Returns'))

### Tableau superstore dataset

In [11]:
try:
    superstore = pd.read_excel('data/superstore.xlsx')
except FileNotFoundError:
    superstore = pd.read_excel('https://query.data.world/s/n2pyux2nabxy4c43zl3uugxsk5gt6v')

In [12]:
superstore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
Row ID            51290 non-null int64
Order ID          51290 non-null object
Order Date        51290 non-null datetime64[ns]
Ship Date         51290 non-null datetime64[ns]
Ship Mode         51290 non-null object
Customer ID       51290 non-null object
Customer Name     51290 non-null object
Segment           51290 non-null object
Postal Code       9994 non-null float64
City              51290 non-null object
State             51290 non-null object
Country           51290 non-null object
Region            51290 non-null object
Market            51290 non-null object
Product ID        51290 non-null object
Category          51290 non-null object
Sub-Category      51290 non-null object
Product Name      51290 non-null object
Sales             51290 non-null float64
Quantity          51290 non-null int64
Discount          51290 non-null float64
Profit            51290 non-null fl

In [13]:
superstore.nunique()

Row ID            51290
Order ID          25728
Order Date         1430
Ship Date          1464
Ship Mode             4
Customer ID       17415
Customer Name       796
Segment               3
Postal Code         631
City               3650
State              1102
Country             165
Region               23
Market                5
Product ID         3788
Category              3
Sub-Category         17
Product Name       3788
Sales             27200
Quantity             14
Discount             29
Profit            28234
Shipping Cost     16753
Order Priority        4
dtype: int64

In [14]:
superstore.Segment.value_counts()

Consumer       26518
Corporate      15429
Home Office     9343
Name: Segment, dtype: int64

### Plot a line chart of sales by segment

#### First, just do one series

In [16]:
# Summarise by month, then group by Segment so we can iterate through the groups
sales = superstore.set_index('Order Date').groupby('Segment').resample('1M')['Sales'].sum().reset_index().groupby('Segment')

In [17]:
# Get just the Consumer segment
group = sales.get_group('Consumer') 

In [18]:
data = [go.Scatter(x=group['Order Date'], y=group['Sales'], name='Consumer')]

In [19]:
py.iplot(data)

#### Now several series

In [20]:
data = [go.Scatter(x=group['Order Date'], y=group['Sales'], name=key) for key, group in sales]

In [21]:
# Set the layout
layout = dict(title = 'Monthly sales by segment',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Sales ($)'),
              )

In [22]:
fig = dict(data=data, layout=layout)
py.iplot(fig)

### Now make the chart interactive - display one line at a time

Based on the plotly examples [here](https://plot.ly/python/dropdowns/)

The `updatemenus` object holds details of the dropdown box we'll use to filter the chart. First, we'll hard-code it - then we'll look at a more automated way.

In [36]:
superstore.Segment.value_counts()

Consumer       26518
Corporate      15429
Home Office     9343
Name: Segment, dtype: int64

In [37]:
updatemenus = list([
    dict(active=3, # Sets the initially active item
         type='buttons',
         direction='right',
         xanchor='left',
         yanchor='top',
         buttons=list([   
            dict(label = 'Consumer',
                 method = 'update',
                 args = [{'visible': [True, False, False]}, {'title': 'Consumer'}]),
            dict(label = 'Corporate',
                 method = 'update',
                 args = [{'visible': [False, True, False]}, {'title': 'Corporate'}]),
            dict(label = 'Home Office',
                 method = 'update',
                 args = [{'visible': [False, False, True]}, {'title': 'Home Office'}]),
            dict(label = 'All',
                 method = 'update',
                 args = [{'visible': [True, True, True]}, {'title': 'All Segments'}]),
        ]),
    )
])


In [38]:
updatemenus

[{'active': 3,
  'type': 'buttons',
  'direction': 'right',
  'xanchor': 'left',
  'yanchor': 'top',
  'buttons': [{'label': 'Consumer',
    'method': 'update',
    'args': [{'visible': [True, False, False]}, {'title': 'Consumer'}]},
   {'label': 'Corporate',
    'method': 'update',
    'args': [{'visible': [False, True, False]}, {'title': 'Corporate'}]},
   {'label': 'Home Office',
    'method': 'update',
    'args': [{'visible': [False, False, True]}, {'title': 'Home Office'}]},
   {'label': 'All',
    'method': 'update',
    'args': [{'visible': [True, True, True]}, {'title': 'All Segments'}]}]}]

In [39]:
updatemenus_working = updatemenus.copy()

In [40]:
# Set the layout
layout = dict(title = 'Monthly sales by segment',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Sales ($)'),
              )
layout['updatemenus'] = updatemenus

In [41]:
fig = dict(data=data, layout=layout)
py.iplot(fig)

### Obviously we should not be hard-coding the values, so here's a more general method:

In [42]:
# Define all elements of updatemenus except for the buttons
updatemenus = list([
    dict(active=0, 
         type='buttons',
         direction='right',
         xanchor='left',
         yanchor='top')
])

In [43]:
# Now add buttons
def make_buttons(menu_items, include_all=True, all_title='All values'):
    """ menu_items = a list of strings, one for each dropdown value. """
    b = []
    
    if include_all:
        b.append(dict(label = "All",
                 method = 'update', 
                 args = [{'visible': [True] * len(menu_items)}, {'title': all_title}])
                )

    for i in menu_items:
        d = dict(label = i,
                 method = 'update', 
                 args = [{'visible': [i == z for z in menu_items]}, {'title': i}])
        b.append(d)
        
    return b

In [44]:
segments = superstore.Segment.unique()
b = make_buttons(segments)

In [45]:
updatemenus[0]['buttons'] = b

In [46]:
updatemenus 

[{'active': 0,
  'type': 'buttons',
  'direction': 'right',
  'xanchor': 'left',
  'yanchor': 'top',
  'buttons': [{'label': 'All',
    'method': 'update',
    'args': [{'visible': [True, True, True]}, {'title': 'All values'}]},
   {'label': 'Consumer',
    'method': 'update',
    'args': [{'visible': [True, False, False]}, {'title': 'Consumer'}]},
   {'label': 'Corporate',
    'method': 'update',
    'args': [{'visible': [False, True, False]}, {'title': 'Corporate'}]},
   {'label': 'Home Office',
    'method': 'update',
    'args': [{'visible': [False, False, True]}, {'title': 'Home Office'}]}]}]

In [47]:
# Set the layout
layout = dict(title = 'Monthly sales by segment',
              xaxis = dict(title = 'Date'),
              yaxis = dict(title = 'Sales ($)'),
              )
layout['updatemenus'] = updatemenus

In [48]:
fig = dict(data=data, layout=layout)
py.iplot(fig)

### Filtering by parent-child relationships

That example demonstrated the principles, but it not really useful in practice. A much more common scenario is where you want to filter a large number of groups down to a smaller set, e.g, display all the sub-categories for a particular product category.

So we will modify our make_buttons function to take a dictionary of parent and child values for menu_items, e.g.

`menu_items = {'cats': ['fluffy', 'tiddles', 'whiskers'], 'dogs': ['rover', 'fido']}`

In [291]:
data = [go.Scatter(x=group['Order Date'], y=group['Sales'], name=key) for key, group in sales]

When creating the button items, we should refer back to the data object, to ensure that we get all the categories in the right order

It probably makes sense to have some sort of metadata object that controls the traces (i.e. each item in the `data` list) and how they relate to the dropdown buttons. Should include a minimunm of:

* trace name
* trace column
* display name (optional)
* parent name (if different)
* parent column (if different)

The function that reads the control object will generate both `data` and `updatemenus`

In [51]:
def get_lookups(df, parent_col, child_col):
    g = df[[parent_col, child_col]].drop_duplicates().groupby(parent_col)
    
    d = dict()
    for key, group in g:
        d[key] = list(group['Sub-Category'])
        
    return d

In [52]:
cats = get_lookups(superstore, 'Category', 'Sub-Category')

In [53]:
cats

{'Furniture': ['Chairs', 'Tables', 'Bookcases', 'Furnishings'],
 'Office Supplies': ['Art',
  'Appliances',
  'Storage',
  'Fasteners',
  'Binders',
  'Labels',
  'Paper',
  'Supplies',
  'Envelopes'],
 'Technology': ['Phones', 'Copiers', 'Machines', 'Accessories']}

In [109]:
def make_chart_data(df, x, y, group_col=None, subgroup_col=None, dropdown=True):
    data = []
    
    # Prepare group vars
    group_vars = []
    if group_col:
        group_vars.append(group_col)
        if subgroup_col is None:
            subgroup_col = group_col
    if subgroup_col:
        group_vars.append(subgroup_col)
        
    print('Group vars = ', group_vars)
    
    # Make data
    if group_col:
        grouped_df = df.groupby(group_vars)
        
        for key, group in grouped_df:
            print(key)
#             data.append(go.Scatter(x=group[x], y=group[y], name=key))

    else:
        data = [1,2,3]
    return data

In [98]:
sales = superstore.set_index('Order Date').groupby('Category').resample('1M')['Sales'].sum().reset_index()

In [99]:
d = make_chart_data(sales, 'Order Date', 'Sales', group_col = 'Category')
d

Group vars =  ['Category']


[Scatter({
     'name': 'Furniture',
     'x': array([datetime.datetime(2012, 1, 31, 0, 0),
                 datetime.datetime(2012, 2, 29, 0, 0),
                 datetime.datetime(2012, 3, 31, 0, 0),
                 datetime.datetime(2012, 4, 30, 0, 0),
                 datetime.datetime(2012, 5, 31, 0, 0),
                 datetime.datetime(2012, 6, 30, 0, 0),
                 datetime.datetime(2012, 7, 31, 0, 0),
                 datetime.datetime(2012, 8, 31, 0, 0),
                 datetime.datetime(2012, 9, 30, 0, 0),
                 datetime.datetime(2012, 10, 31, 0, 0),
                 datetime.datetime(2012, 11, 30, 0, 0),
                 datetime.datetime(2012, 12, 31, 0, 0),
                 datetime.datetime(2013, 1, 31, 0, 0),
                 datetime.datetime(2013, 2, 28, 0, 0),
                 datetime.datetime(2013, 3, 31, 0, 0),
                 datetime.datetime(2013, 4, 30, 0, 0),
                 datetime.datetime(2013, 5, 31, 0, 0),
                 datetime

In [70]:
sales.head()

,Category,Order Date,Sales
0,Furniture,2012-01-31,34463.7530
1,Furniture,2012-02-29,35799.3875
2,Furniture,2012-03-31,40277.2748
3,Furniture,2012-04-30,30689.5451
4,Furniture,2012-05-31,49768.6720


In [102]:
sales = (superstore
         .set_index('Order Date')
         .groupby(['Category', 'Sub-Category'])
         .resample('1M')['Sales']
         .sum()
         .reset_index()
        )

In [107]:
gb = sales.groupby(['Category', 'Category'])

In [108]:
for key, value in gb:
    print(key)

('Furniture', 'Furniture')
('Office Supplies', 'Office Supplies')
('Technology', 'Technology')
